# Elementos da versão

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

Tivemos problemas de análise dos resultados da v1. Os dados de mortalidade de doenças evitáveis em 2022 por município estão muito altos e fora do padrão esperado.

O objetivo desta v.2 é criar um dataset diferente, mais básico que permite que aprendamos com os erros e acertos, verificando melhor cada etapa de trabalho.

O dataset desta v.2 deverá ter as seguintes características:

* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: CID evitáveis
* subgroup: municípios
* estimate: mort-avoid por município
* setting-average: mort-avoid por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




#### Aspectos gerais e participantes

Projeto Desenvolvido em parceria da MS/SECTICS/DESID com a Organização Mundial de Saúde

Elaborado em linguagem Python, utilizando a biblioteca pysus para acesso ao SIM (Sistema de Informações de Mortalidade do SUS)

Coordenador-geral:

Coordenadores: 

Cientista de dados: Marcos Mesquita

Outros participantes:

Data: 06/12/2024


#### Objetivos

O principal objetivo desse projeto é realizar a avaliação de desigualdade na saúde no Brasil com ênfase para a dimensão de unidades subnacionais, estados e municípios brasileiros, a partir da aplicação do modelo do HEAT, tendo como objetivos específicos:

- Apresentar indicadores de saúde no Brasil desagregados por subunidades nacionais em uma visualização interativa a partir do modelo do HEAT Plus;
- Oferecer um repositório comum e atualizado com dados de desigualdade da saúde no Brasil desagregados por estados e municípios em um modelo sob reconhecimento internacional;
- Permitir exploração e acesso dos dados e da metodologia para outros tipos de análises.

# Etapa 1 - Definição do Bloco

O projeto ocorrerá em blocos sucessivos, nos quais serão definidos o problema e os indicadores a serem analisados.

### Definição do problema do bloco



 H0 - as unidades federativa são variáveis determinantes para mortalidade no país

### Escopo do Bloco 1


* indicador: mortalidade (número de mortes por população, dados de morte do SIM/SUS e dados populacionais pelo Censos de 2010 e 2022)
* localização: todo o Brasil, com agregação de mortalidade por Estados e Municípios
* período: 2010 e 2022 
* doenças: “doenças evitáveis” conforme definido pelo OCDE
* outros critérios de filtro: até 74 anos de idade (em razão da aplicação do critério de "doenças evitáveis")

# Etapa 2 - Análise primária dos dados por bloco

## Aspectos teóricos e técnicos das fontes


#### Aspectos técnicos - ambiente, linguagem e ferramentas principais

A análise será feita utilizando a linguagem Python e a biblioteca Pysus que permite acesso às principais bases do SUS de forma direta.
O Pysus exige um ambiente linux para execução de alguns comandos o que foi possível, mesmo em uma máquina Windows, a partir do WSL do Windows.
Foi criado um ambiente em WSL e o conjunto da extração, tratamento e análise dos dados foi realizado no formato Notebook em um editor VSCode, dentro de um Dev Container: Default Linux Universal.


#### Morte e mortalidade no HEAT

Para entendermos como o HEAT trata a mortalidade, fizemos uma busca de indicadores de mortalidade no WHO Health Inequality Data Repository.

O repositório traz dados e metadados de todos os indicadores utilizados pelo HEAT.
Apresenta um total de 3915 indicadores, distribuídos entre 14 seções.

Foi feita uma revisão geral dos indicadores relacionados à mortalidade, que depois foi refinada pela busca das seguintes palavras : “death”, “mortality”, “fatal”, ‘fatality” ,“Probability of dying”, “suicide”. E foram encontrados 388 indicadores que trazem uma destas expressões na sua descrição.


#### SIM (Sistema de Informações de Mortalidade do SUS)

A mortalidade e os indicadores de morte serão buscados no SIM (Sistema de Informações de Mortalidade do SUS).
O SIM disponibiliza dados de mortes no Brasil desde 1996 por ocorrências, gerido pelo Departamento de Análise de Situação de Saúde, da Secretaria de Vigilância em Saúde do Ministério da Saúde, em conjunto com as Secretarias Estaduais e Municipais de Saúde. Estas últimas responsáveis efetivamente pela coleta dos dados.

O SIM traz informações do indivíduo, como sexo, idade, localidade e causas da morte, classificadas de acordo com o CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde).


#### CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde)

Com o intuito de padronizar a nomenclatura de patologias e com abrangência para todo o mundo, foi elaborado pela Organização Mundial da Saúde (OMS) uma  classificação internacional de doenças, a CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde). 

A Classificação é atualizada periodicamente. A versão mais atual é a CID11, publicada em janeiro de 2022.

No entanto, por se tratar de revisão que traz impactos nos registros e que exige considerável atualização das ferramentas de captura das informações, o SIM utiliza versões anteriores da CID.
- CID9 - dados até 1995
- CID10 - a partir de 1996 até a data atual

Em razão do recorte temporal dos dados, de 1998 a 2022, iremos utilizar somente a CID10.
A CID10 é organizada em 22 capítulos e possui, além da classificação de doenças, descrição de códigos que permitem identificar sinais, sintomas, queixas, causas externas e circunstâncias sociais.


#### CID de doenças evitáveis

Como elemento do escopo deste trabalho iremos tratar de doenças evitáveis.

O Brasil, a partir da Secretaria de Vigilância em Saúde apresenta uma classificação de doenças evitáveis. 

Lista de causas evitáveis pode ser encontrada nos links: http://tabnet.datasus.gov.br/cgi/sim/Obitos_Evitaveis_0_a_4_anos.pdf




#### População (Censo)


Os dados de população são os do Censo, acessados pelo site do IBGE, https://sidra.ibge.gov.br/Tabela/9606






### Instalação e importação de bibliotecas

#### Instalação de bibliotecas

In [ ]:
# Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
!apt-get update  
!apt-get install libffi-dev  # Install libffi-dev
!pip install --upgrade pip  # Ensure pip is up to date
!pip install pysus # Exa
!pip install gdown
!pip install pylance
!pip install tabulate

#### Importação de bibliotecas

In [2]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate


In [27]:
# importar bibliotecas para carga dos dados
import pysus
# import pylance
from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

In [24]:
# Para organizar resultados, gerar prints em amarelo

def print_y(text):
  """Imprime o texto fornecido na cor amarela.

  Args:
    text: O texto a ser impresso.
  """
  print(f"\033[33m ==> {text}\033[0m")

# Exemplo de uso:
print_y("Este texto será impresso em amarelo!")
print("Este em normal!")

 ==> Este texto será impresso em amarelo!
Este em normal!


#### Diretórios - configuração

In [ ]:
# Definir diretório padrão para pysus
pysus.online_data.SIM.CACHEPATH="./downloads_sim_original"
# Definir pasta para download de arquivos csv e parquet
import os 

# Define diretório para armazenar arquivos do sim (carregados por pysus) quando o SIM.CACHEPATH não funcionar
os.makedirs('downloads_sim_original', exist_ok=True)  
downloads_sim = "downloads_sim_original"

# Define diretório para armazenar outros arquivos
os.makedirs('downloads_outros_v2', exist_ok=True)  

# Define diretório para armazenar arquivos trabalhados
os.makedirs('files_clean_v2', exist_ok=True) 


# Define diretório para exportar arquivos finais ou de relatórios
os.makedirs('exportar_v2', exist_ok=True) 

#### Acesso ao SIM

In [4]:
sim = SIM().load() # Loads the files from DATASUS

In [5]:
sim.paths

(/dissemin/publicos/SIM/CID10/DORES, /dissemin/publicos/SIM/CID9/DORES)

In [6]:
sim.groups

{'CID10': 'DO', 'CID9': 'DOR'}

## Arquivo principal - DO

DO - Declaração de Óbito, conforme acessado no SIM

#### Carga SIM principal

In [32]:
# Carrega os dados do SIM para o ano de 2010
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "" # A sigla BR carrega dados de todos as unidades federativas
ano = [2010] # Anos do escopo, 2010 e 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

# sim.get_files("CID10", uf="BR", year=2010)
dobr_2010 = download(grupo,uf,ano,downloads_sim) # Carrega os dados do SIM
# dobr2010 = download("CID10","BR",2010,'TESTE_EXCLUIR') # Carrega os dados do SIM
# dobr2010 

DOTO2010.parquet: 100%|██████████| 16.6k/16.6k [00:00<00:00, 51.3kB/s]


In [8]:
# Carrega os dados do SIM para os demais anos
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "BR" # A sigla BR carrega dados de todos as unidades federativas
ano = [2019,2020,2021,2022] # Anos do escopo, 2010 e 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

# sim.get_files("CID10", uf="BR", year=2010)
dobr2010 = download(grupo,uf,ano,downloads_sim) # Carrega os dados do SIM
# dobr2010 = download("CID10","BR",2010,'TESTE_EXCLUIR') # Carrega os dados do SIM
# dobr2010 

0it [00:00, ?it/s]


In [ ]:
# dodf = dobr.to_dataframe()

### Tratamento do arquivo principal

O dicionário do SIM está em "./Estrutura_SIM_para_CD.pdf"

Trabalhamos o CID presente no campo CAUSABAS.

Em razão do escopo da População Estimada (que apresenta dados por Município, mas não de outras variáveis de desigualdade) a primeira versão será um recorte da base DOBR com os dados de CID ('CAUSABAS'), idade (a partir do tratamento de 'DTNASC'), e de Município (a partir do tratamento de 'CODMUNRES').

Por se tratar de doenças evitáveis, o primeiro filtro a ser feito é com base na idade de óbito, que deve ser abaixo de 75 anos.

A idade de óbito está presente no campo IDADE. Ela é composta de 3 dígitos, o primeiro identifica se a contagem é por dias(1), semanas(2), meses (3), ano (4), mais de 100 anos (5).

Em razão disto, vamos desconsiderar todos os valores de dígito 4 e mais de 75 (475).


Iremos tratar do arquivo de 2010 separado dos outros anos pela especificade daquele arquivo.



In [13]:
# Acessar os dados do SIM de 2010 como 1 dataframe

# Definir o diretório onde os arquivos estão localizados
directory = 'downloads_sim_original'

# Listar todos os arquivos na pasta
file_list = os.listdir(directory)

# Filtrar apenas os arquivos parquet
parquet_files = [file for file in file_list if file.endswith('2010.parquet')]

# Inicializar uma lista para armazenar os dataframes
dataframes = []

# Iterar sobre os arquivos parquet e ler cada um em um dataframe
for file in parquet_files:
    file_path = os.path.join(directory, file)
    df = pd.read_parquet(file_path)
    dataframes.append(df)

# Concatenar todos os dataframes em um único dataframe
dobr_10_temp = pd.concat(dataframes, ignore_index=True)

# Exibir informações do dataframe combinado
dobr_10_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136947 entries, 0 to 1136946
Data columns (total 60 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CONTADOR    1133938 non-null  object
 1   ORIGEM      1136947 non-null  object
 2   TIPOBITO    1136947 non-null  object
 3   DTOBITO     1136947 non-null  object
 4   HORAOBITO   1136947 non-null  object
 5   NATURAL     1136947 non-null  object
 6   DTNASC      1136947 non-null  object
 7   IDADE       1136947 non-null  object
 8   SEXO        1136947 non-null  object
 9   RACACOR     1136947 non-null  object
 10  ESTCIV      1136947 non-null  object
 11  ESC         1136947 non-null  object
 12  OCUP        1136947 non-null  object
 13  CODMUNRES   1136947 non-null  object
 14  CODBAIRES   1136947 non-null  object
 15  LOCOCOR     1136947 non-null  object
 16  CODESTAB    1136947 non-null  object
 17  CODMUNOCOR  1136947 non-null  object
 18  CODBAIOCOR  1136947 non-null  object
 19  

In [14]:
# Excluir valores nulos da coluna CAUSABAS
dobr_10_temp01 = dobr_10_temp.dropna(subset=['CAUSABAS'])
dobr_10_temp01.shape[0]


1136947

In [15]:
# Reduzir as colunas
dobr_10_temp02 = dobr_10_temp01[['DTOBITO','CAUSABAS', 'CODMUNRES','IDADE']]

# Exibir informações do dataframe após a exclusão
dobr_10_temp02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136947 entries, 0 to 1136946
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   DTOBITO    1136947 non-null  object
 1   CAUSABAS   1136947 non-null  object
 2   CODMUNRES  1136947 non-null  object
 3   IDADE      1136947 non-null  object
dtypes: object(4)
memory usage: 34.7+ MB


In [16]:
# Arquivo para doenças evitáveis - excluir maiores de 74 anos
dobr_10_temp03 = dobr_10_temp02.copy()

# Substituir valores '     ' e '-' por '0' na coluna IDADE
dobr_10_temp03['IDADE'] = dobr_10_temp03['IDADE'].replace(['     '], '0')

# transformar IDADE em inteiro e excluir valores maiores que 474
dobr_10_temp03['IDADE'] = dobr_10_temp03['IDADE'].astype(int)

dobr_10_temp03.shape[0]


1136947

In [18]:
# Filtrar as idades menores de 75
dobr_10_temp04 = dobr_10_temp03[dobr_10_temp03['IDADE'] < 475].reset_index(drop=True)
dobr_10_temp04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722229 entries, 0 to 722228
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   DTOBITO    722229 non-null  object
 1   CAUSABAS   722229 non-null  object
 2   CODMUNRES  722229 non-null  object
 3   IDADE      722229 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 22.0+ MB


In [2]:
# Carregar e tratar arquivos de DO de outros anos

# Read the parquet files
dobr2019 = pd.read_parquet('downloads_sim_original/DOBR2019.parquet')
dobr2020 = pd.read_parquet('downloads_sim_original/DOBR2020.parquet')
dobr2021 = pd.read_parquet('downloads_sim_original/DOBR2021.parquet')
dobr2022 = pd.read_parquet('downloads_sim_original/DOBR2022.parquet')

# Concatenate the dataframes
dobr_19a22_temp = pd.concat([dobr2019,dobr2020,dobr2021,dobr2022], ignore_index=True)

In [4]:
dobr_19a22_temp.shape[0]

6283540

In [6]:
# Filtrar campos
dobr_19a22_temp01 = dobr_19a22_temp[['DTOBITO','CAUSABAS', 'CODMUNRES','IDADE']]
dobr_19a22_temp01.shape[0]

6283540

In [11]:
# transformar IDADE em inteiro 
dobr_19a22_temp02 = dobr_19a22_temp01.copy()
dobr_19a22_temp02['IDADE'] = dobr_19a22_temp02['IDADE'].astype(int)


In [12]:
# Excluir valores maiores que 474
dobr_19a22_temp03 = dobr_19a22_temp02[dobr_19a22_temp02['IDADE'] < 475]

# Display the combined dataframe
dobr_19a22_temp03.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3724693 entries, 1 to 6283539
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   DTOBITO    object
 1   CAUSABAS   object
 2   CODMUNRES  object
 3   IDADE      int64 
 4   IDADE_     int64 
dtypes: int64(2), object(3)
memory usage: 170.5+ MB


In [19]:
# Concatenate the dataframes
dobr_10_19a22_temp = pd.concat([dobr_19a22_temp03,dobr_10_temp04], ignore_index=True)
dobr_10_19a22_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446922 entries, 0 to 4446921
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   DTOBITO    object 
 1   CAUSABAS   object 
 2   CODMUNRES  object 
 3   IDADE      int64  
 4   IDADE_     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 169.6+ MB


In [25]:
# Resultado

shape1 = dobr_10_19a22_temp.CAUSABAS.count()

print_y(f" O arquivo de declarações de óbito dos anos de 2010,2019,2020,2021 e 2022 tem o total de {shape1} registros ")


 ==>  O arquivo de declarações de óbito dos anos de 2010,2019,2020,2021 e 2022 tem o total de 4446922 registros 


In [22]:
# Ajustar os campos
dobr_10_19a22_temp01 = dobr_10_19a22_temp.copy()

# assegurar CODMUNRES como string
dobr_10_19a22_temp01['CODMUNRES'] = '_' + dobr_10_19a22_temp01['CODMUNRES'].astype(str).str[:6]

# campo ano_obito formado pelos 4 últimos caracteres de DTOBITO, como número
dobr_10_19a22_temp01['ANO_OBITO'] = '_' + dobr_10_19a22_temp01['DTOBITO'].str[-4:]

# excluir colunas DTOBITO e IDADE
dobr_10_19a22_temp02 = dobr_10_19a22_temp01.drop(['DTOBITO','IDADE','IDADE_'], axis=1)

# Exibir as primeiras linhas do DataFrame resultante
dobr_10_19a22_temp02.head()

,CAUSABAS,CODMUNRES,ANO_OBITO
0,I219,_292740,_2019
1,I10,_355100,_2019
2,X700,_500630,_2019
3,C539,_500500,_2019
4,V220,_500627,_2019


#### Resultados e geração de arquivo DOBR

In [23]:
# Salvar arquivo em parquet
dobr_10_19a22_temp02.to_parquet('files_clean/dobr_10_19a22.parquet')

In [29]:
# Contador

# apresentar valor da diferença entre os números de registros de dobr e dobr_filtered

shape_2010 = dobr_10_temp.shape[0]
shape_2019 = dobr2019.shape[0]
shape_2020 = dobr2020.shape[0]
shape_2021 = dobr2021.shape[0]
shape_2022 = dobr2022.shape[0]
shape_all = shape_2010 + shape_2019 + shape_2020 + shape_2021 + shape_2022

print_y(f"""O número de registros de óbitos originais por ano:
        * 2010: {shape_2010}
        * 2019: {shape_2019}
        * 2020: {shape_2020}
        * 2021: {shape_2021}
        * 2022: {shape_2022}
        """)


dif = shape_all - dobr_10_19a22_temp02.shape[0]
print(f"\033[33m ==> O número de registros excluídos por apresentar idade maior de 74 anos foi de {dif} registros\033[0m")
print(f"\033[33m ==> O número de registros de óbitos resultantes é de {dobr_10_19a22_temp02.shape[0]} registros\033[0m")

 ==> O número de registros de óbitos originais por ano:
        * 2010: 1136947
        * 2019: 1349801
        * 2020: 1556824
        * 2021: 1832649
        * 2022: 1544266
        
 ==> O número de registros excluídos por apresentar idade maior de 74 anos foi de 2973565 registros
 ==> O número de registros de óbitos resultantes é de 4446922 registros


## Carga SIM auxiliar

O SIM traz algumas tabelas auxiliares de CID10 e municípios. 

Não iremos trazemos estas informações, no entanto, porque iremos utilizar dados de municípios concomitante à população do Censo do IBGE e sobre os CIDs iremos consultar a informações já da lista de doenças evitáveis, conforme OCDE.

In [ ]:
# # Download de tabela auxiliar de CID10
# cid10 = get_CID10_table()
# cid10.head()

In [ ]:
# # Download de tabela auxiliar de CID10
# munic = get_municipios()
# print(munic.head())

## Carga externa ao SIM

Iremos carregar informações de fontes externas sobre população e CIDs de doenças evitáveis.

##### Carga de CIDs doenças evitáveis - SVS

In [19]:
# # Download de tabela tratada de CIDs evitáveis de acordo com documentos da SVS

# url = 'https://docs.google.com/spreadsheets/d/1YWurDqnBDMz4ACOEctY9uuQkEm7uSRAogzZETQrLCXk/edit?usp=drive_link'
# # Convert the Google Sheets URL to a CSV export URL
# csv_url = url.replace('/edit?usp=drive_link', '/export?format=csv')

# # Read the CSV data into a pandas DataFrame
# df = pd.read_csv(csv_url)

# # Now you can work with the DataFrame 'df'
# df.head()


In [20]:
# # Salvar arquivo csv
# df.to_csv('downloads_outros/cid10_evit_svs.csv', index=False)

##### Carga de CIDs doenças evitáveis - OCDE

In [ ]:
# # Download de tabela tratada de CIDs evitáveis de acordo com documentos da OCDE
# url = 'https://docs.google.com/spreadsheets/d/1X8AH0zs4PNc_bUpZ3tBvFw2HGpgb9CjShOkwXQ0so_I/edit?usp=sharing'
# # Convert the Google Sheets URL to a CSV export URL
# csv_url = url.replace('/edit?usp=sharing', '/export?format=csv')

# # Read the CSV data into a pandas DataFrame
# ocde_evit = pd.read_csv(csv_url)

# # Now you can work with the DataFrame 'df'
# ocde_evit.head()

,avoid_flag,prevent_flag,treat_flag,Group,Causes of deaths,Rationale for inclusion,Range,cid_pai
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [23]:
ocde_evit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


## População Censo

### Carga de População Censo 


Os dados populacionais são os dos Censos de 2010 e 2022.

Nesta v2, não iremos tratar com variáveis de sexo, raça e faixa etária (para calcular menores de 75 anos).

A fonte de dados para gerar os arquivos ou o endereço do caminho SIDRA, é https://sidra.ibge.gov.br/Tabela/9605.

Nota da v.2 : iremos trazer os dados do Censo sem precisar desagregar por sexo e raça-cor e consideramos que não será necessário trazer dados por UF, estes podem ser agregados depois a partir dos dados de município.

In [72]:
# # Definir a URL
# url = 'https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela9606.csv&terr=NCS&rank=-&query=t/9606/n3/all/n6/all/v/allxp/p/all/c86/95251/c2/6794/c287/6653,49108,49109,60040,60041,93098,100362/l/,v%2Bc287,p%2Bt%2Bc86%2Bc2&verComplementos2=false&verComplementos1=false&omitirIndentacao=false&abreviarRotulos=false&exibirNotas=false&agruparNoCabecalho=false'

# # Ler os dados da URL em um DataFrame
# # censo_pop = pd.read_json(url)
# ibge_download_ = pd.read_csv(url, skiprows=3, dtype=str)

# # Exibir as primeiras linhas do DataFrame
# ibge_download_.head()

,Ano,Nível,Cód.,Unidade da Federação e Município,Cor ou raça,Sexo,Total,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais
0,2010,UF,11,Rondônia,Total,Total,1562409,13015,7291,3170,1126,330,82
1,2010,UF,12,Acre,Total,Total,733559,5508,3574,1790,737,300,91
2,2010,UF,13,Amazonas,Total,Total,3483985,24686,14803,7829,3405,1249,415
3,2010,UF,14,Roraima,Total,Total,450479,2575,1586,809,324,100,35
4,2010,UF,15,Pará,Total,Total,7581051,62725,37009,18075,8912,3349,899


In [73]:
# Exibir informaçãoes gerais do DataFrame
ibge_download_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11214 entries, 0 to 11213
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Ano                               11213 non-null  object
 1   Nível                             11201 non-null  object
 2   Cód.                              11194 non-null  object
 3   Unidade da Federação e Município  11194 non-null  object
 4   Cor ou raça                       11194 non-null  object
 5   Sexo                              11194 non-null  object
 6   Total                             11194 non-null  object
 7   75 a 79 anos                      11194 non-null  object
 8   80 a 84 anos                      11194 non-null  object
 9   85 a 89 anos                      11194 non-null  object
 10  90 a 94 anos                      11194 non-null  object
 11  95 a 99 anos                      11194 non-null  object
 12  100 anos ou mais  

In [74]:
# Primeira linha de tratamento para salvar arquivo

# Excluir notas e comentários do arquivo
ibge_download = ibge_download_.dropna(subset=['Total']).reset_index(drop=True)

# Exibir as primeiras linhas do DataFrame
ibge_download.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11194 entries, 0 to 11193
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Ano                               11194 non-null  object
 1   Nível                             11194 non-null  object
 2   Cód.                              11194 non-null  object
 3   Unidade da Federação e Município  11194 non-null  object
 4   Cor ou raça                       11194 non-null  object
 5   Sexo                              11194 non-null  object
 6   Total                             11194 non-null  object
 7   75 a 79 anos                      11194 non-null  object
 8   80 a 84 anos                      11194 non-null  object
 9   85 a 89 anos                      11194 non-null  object
 10  90 a 94 anos                      11194 non-null  object
 11  95 a 99 anos                      11194 non-null  object
 12  100 anos ou mais  

In [80]:
# Segunda linha de tratamento para salvar arquivo

# Excluir colunas de "Cor ou raça" e "Sexo"
ibge_download01 = ibge_download.drop(columns=['Sexo','Cor ou raça'])

# Renomear colunas
ibge_download01.rename(columns={'Unidade da Federação e Município': 'unidade_no'}, inplace=True)

# Exibir informações gerais do dataframe
ibge_download01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11194 entries, 0 to 11193
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Ano               11194 non-null  object
 1   Nível             11194 non-null  object
 2   Cód.              11194 non-null  object
 3   unidade_no        11194 non-null  object
 4   Total             11194 non-null  object
 5   75 a 79 anos      11194 non-null  object
 6   80 a 84 anos      11194 non-null  object
 7   85 a 89 anos      11194 non-null  object
 8   90 a 94 anos      11194 non-null  object
 9   95 a 99 anos      11194 non-null  object
 10  100 anos ou mais  11194 non-null  object
dtypes: object(11)
memory usage: 962.1+ KB


In [ ]:
# Salvar arquivo de população do Censo
ibge_download01.to_parquet('downloads_outros/censo_pop_raw.parquet')

### Tratamento de População Censo Municípios

In [30]:
# Carga de arquivo de População do Censo
censo_pop_temp = pd.read_parquet('downloads_outros/censo_pop_raw.parquet')

In [31]:
censo_pop_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11194 entries, 0 to 11193
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Ano               11194 non-null  object
 1   Nível             11194 non-null  object
 2   Cód.              11194 non-null  object
 3   unidade_no        11194 non-null  object
 4   Total             11194 non-null  object
 5   75 a 79 anos      11194 non-null  object
 6   80 a 84 anos      11194 non-null  object
 7   85 a 89 anos      11194 non-null  object
 8   90 a 94 anos      11194 non-null  object
 9   95 a 99 anos      11194 non-null  object
 10  100 anos ou mais  11194 non-null  object
dtypes: object(11)
memory usage: 962.1+ KB


In [46]:
censo_pop_temp.head()

,Ano,Nível,Cód.,unidade_no,Total,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais
0,2010,UF,11,Rondônia,1562409,13015,7291,3170,1126,330,82
1,2010,UF,12,Acre,733559,5508,3574,1790,737,300,91
2,2010,UF,13,Amazonas,3483985,24686,14803,7829,3405,1249,415
3,2010,UF,14,Roraima,450479,2575,1586,809,324,100,35
4,2010,UF,15,Pará,7581051,62725,37009,18075,8912,3349,899


In [36]:
# Identificar a classificação de unidades federativas
print(f'Unidades federativas do arquivo Censo_pop: {censo_pop_temp.Nível.unique()}')

# Apresentar os anos do Censo
print(f'Anos do arquivo Censo_pop: {censo_pop_temp.Ano.unique()}')

Unidades federativas do arquivo Censo_pop: ['UF' 'MU']
Anos do arquivo Censo_pop: ['2010' '2022']


In [ ]:
# # Filtrar os dados apenas de município
# censo_pop_mun = censo_pop_raw[censo_pop_raw['Nível'] == 'MU']
# censo_pop_mun.info()

In [37]:
# Preparar colunas para conseguir valores da população até 74 anos

# Colunas para transformar em inteiro
cols_age = ['Total','75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
# para todas as colunas identificar substituir '-' por 0
censo_pop_temp01 = censo_pop_temp.copy()
censo_pop_temp01[cols_age] = censo_pop_temp01[cols_age].replace('-',0)

# transformar todas as colunas definidas para formato integer
censo_pop_temp01[cols_age] = censo_pop_temp01[cols_age].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
censo_pop_temp01.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11194 entries, 0 to 11193
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Ano               11194 non-null  object
 1   Nível             11194 non-null  object
 2   Cód.              11194 non-null  object
 3   unidade_no        11194 non-null  object
 4   Total             11194 non-null  int64 
 5   75 a 79 anos      11194 non-null  int64 
 6   80 a 84 anos      11194 non-null  int64 
 7   85 a 89 anos      11194 non-null  int64 
 8   90 a 94 anos      11194 non-null  int64 
 9   95 a 99 anos      11194 non-null  int64 
 10  100 anos ou mais  11194 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 962.1+ KB


In [39]:
# Gerar coluna de população total até 74 anos
cols_age_over75 = ['75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']

censo_pop_temp02 = censo_pop_temp01.copy()

# Subtrair a população de 75 anos ou mais da população total
censo_pop_temp02['pop_under74'] = censo_pop_temp02['Total'] - censo_pop_temp02[cols_age_over75].sum(axis=1)

# Excluir colunar cols_age_over75 e 0 a 4 anos
censo_pop_temp02.drop(columns=cols_age_over75, inplace=True)

# Renomear coluna Total para pop_total
censo_pop_temp02.rename(columns={'Total': 'pop_total'}, inplace=True)

# mostrar dados gerais da tabela após transformações
print(censo_pop_temp02.shape[0])
censo_pop_temp02.head()

11194


,Ano,Nível,Cód.,unidade_no,pop_total,pop_under74
0,2010,UF,11,Rondônia,1562409,1537395
1,2010,UF,12,Acre,733559,721559
2,2010,UF,13,Amazonas,3483985,3431598
3,2010,UF,14,Roraima,450479,445050
4,2010,UF,15,Pará,7581051,7450082


In [47]:
# Gerar informação de UF a partir do split de 'Município', com o dividor '('
censo_pop_temp03 = censo_pop_temp02.copy()
censo_pop_temp03['UF'] = censo_pop_temp03['unidade_no'].str.split('(', expand=True)[1].str.replace(')','')
censo_pop_temp03['uf_co'] = '_' + censo_pop_temp03['Cód.'].astype(str).str[:2]
# censo_pop_temp03['unidade_no'] = censo_pop_temp03['unidade_no'].str.split('(', expand=True)[0] # Permancer o código de UF depois do nome do município

# mostrar dados gerais da tabela após transformações
print(censo_pop_temp03.shape[0])
censo_pop_temp03.head()


11194


,Ano,Nível,Cód.,unidade_no,pop_total,pop_under74,UF,uf_co
0,2010,UF,11,Rondônia,1562409,1537395,None,_11
1,2010,UF,12,Acre,733559,721559,None,_12
2,2010,UF,13,Amazonas,3483985,3431598,None,_13
3,2010,UF,14,Roraima,450479,445050,None,_14
4,2010,UF,15,Pará,7581051,7450082,None,_15


In [59]:
# Dividir arquivos em UF e Municípios

# Criar arquivo de UF
censo_pop_uf_temp = censo_pop_temp03[censo_pop_temp03['Nível'] == 'UF'].reset_index(drop=True)

# Criar arquivo de Municípios
censo_pop_mun_temp = censo_pop_temp03[censo_pop_temp03['Nível'] == 'MU'].reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
print(f'Registro da tabela censo_pop_uf_temp: {censo_pop_uf_temp.shape[0]}')
print(tabulate(censo_pop_uf_temp.head(), headers='keys', tablefmt='psql'))
print(f'Registro da tabela censo_pop_mun_temp: {censo_pop_mun_temp.shape[0]}')
print(tabulate(censo_pop_mun_temp.head(), headers='keys', tablefmt='psql'))

Registro da tabela censo_pop_uf_temp: 54
+----+-------+---------+--------+--------------+-------------+---------------+------+---------+
|    |   Ano | Nível   |   Cód. | unidade_no   |   pop_total |   pop_under74 | UF   | uf_co   |
|----+-------+---------+--------+--------------+-------------+---------------+------+---------|
|  0 |  2010 | UF      |     11 | Rondônia     |     1562409 |       1537395 |      | _11     |
|  1 |  2010 | UF      |     12 | Acre         |      733559 |        721559 |      | _12     |
|  2 |  2010 | UF      |     13 | Amazonas     |     3483985 |       3431598 |      | _13     |
|  3 |  2010 | UF      |     14 | Roraima      |      450479 |        445050 |      | _14     |
|  4 |  2010 | UF      |     15 | Pará         |     7581051 |       7450082 |      | _15     |
+----+-------+---------+--------+--------------+-------------+---------------+------+---------+
Registro da tabela censo_pop_mun_temp: 11140
+----+-------+---------+---------+----------------

In [62]:
# Preparar para salvar arquivo de UF
censo_pop_uf_temp01 = censo_pop_uf_temp.copy()

# Excluir colunas desnecessárias
censo_pop_uf_temp01.drop(columns=['Cód.','Nível','UF'], inplace=True)

# Renomear coluna unidade_no para uf_no
censo_pop_uf_temp01.rename(columns={'unidade_no': 'uf_no'}, inplace=True)

# mostrar dados gerais da tabela após transformações
print(f'Registro da tabela : {censo_pop_uf_temp01.shape[0]}')
print(tabulate(censo_pop_uf_temp01.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 54
+----+-------+----------+-------------+---------------+---------+
|    |   Ano | uf_no    |   pop_total |   pop_under74 | uf_co   |
|----+-------+----------+-------------+---------------+---------|
|  0 |  2010 | Rondônia |     1562409 |       1537395 | _11     |
|  1 |  2010 | Acre     |      733559 |        721559 | _12     |
|  2 |  2010 | Amazonas |     3483985 |       3431598 | _13     |
|  3 |  2010 | Roraima  |      450479 |        445050 | _14     |
|  4 |  2010 | Pará     |     7581051 |       7450082 | _15     |
+----+-------+----------+-------------+---------------+---------+


In [63]:
# Preparar para salvar arquivo de Municipios
censo_pop_mun_temp01 = censo_pop_mun_temp.copy()

# Mudar o formato de mun_cod para string e extrair os primeiros 7 dígitos
censo_pop_mun_temp01['mun_cod_sim'] = censo_pop_mun_temp01['Cód.'].astype(str).str[:6]

# Acrescentar um '_' no início do valor da coluna mun_cod
censo_pop_mun_temp01['mun_cod_sim'] = '_' + censo_pop_mun_temp01['mun_cod_sim'].astype(str)

# Renomear coluna unidade_no para uf_no
censo_pop_mun_temp01.rename(columns={'unidade_no': 'mun_no'}, inplace=True)

# Excluir colunas desnecessárias
censo_pop_mun_temp01.drop(columns=['Cód.','Nível'], inplace=True)

# Ordenar as colunas
censo_pop_mun_temp02 = censo_pop_mun_temp01[['Ano','uf_co','UF','mun_cod_sim','mun_no','pop_total','pop_under74']]

# mostrar dados gerais da tabela após transformações
print(f'Registro da tabela : {censo_pop_mun_temp02.shape[0]}')
print(tabulate(censo_pop_mun_temp02.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 |
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+


#### Arquivo pronto - censo por municípios e por UF

In [51]:
# Arquivo de População Censo Tratado (pronto para merge)
censo_pop_mun_temp02.to_parquet('files_clean/censo_pop_mun.parquet')

In [52]:
# Salvar arquivo de População do Censo uf
censo_pop_uf_temp01.to_parquet('files_clean/censo_pop_uf.parquet')

## Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [64]:
ocde_evit_temp = pd.read_csv('downloads_outros/cid10_evit_ocde.csv')
ocde_evit_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [65]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_clean/dobr_10_19a22.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


In [67]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit_temp
column_b = 'cid_pai'
column_return = 'avoid_flag'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(tabulate(df_a))

-----  ----  -------------------
    0  I219  Avoidable mortality
    1  I10   Avoidable mortality
    2  X700  Avoidable mortality
    3  C539  causa não evitável
    4  V220  Avoidable mortality
    5  W019  Avoidable mortality
    6  X709  Avoidable mortality
    7  G809  causa não evitável
    8  G309  causa não evitável
    9  J440  Avoidable mortality
   10  B207  Avoidable mortality
   11  J449  Avoidable mortality
   12  V899  Avoidable mortality
   13  I110  Avoidable mortality
   14  L030  causa não evitável
   15  J189  Avoidable mortality
   16  I619  Avoidable mortality
   17  C480  causa não evitável
   18  J850  Avoidable mortality
   19  C61   causa não evitável
   20  N390  causa não evitável
   21  A419  causa não evitável
   22  C229  Avoidable mortality
   23  C920  causa não evitável
   24  N170  Avoidable mortality
   25  I340  causa não evitável
   26  C189  causa não evitável
   27  C23   causa não evitável
   28  X954  Avoidable mortality
   29  V489  Avoidable

In [68]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4446922
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  3 | C539       | _500500     | _2019       | causa não evitável  |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


In [83]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('classificacao').count()

,CAUSABAS,CODMUNRES,ANO_OBITO
classificacao,,,
Avoidable mortality,2537888,2537888,2537888
causa não evitável,1909034,1909034,1909034


In [84]:
# Manter na base só dados de CID evitáveis
dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp02
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
|  5 | W019       | _500325     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


#### Resultados do merge de cid_avoid e dobr

In [85]:
# Salvar arquivo final de DOBR
dobr_evit_merged_temp02.to_parquet('files_clean/dobr_evit_merged_v2.parquet')

In [87]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp02.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f" O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos ")
print_y(f" O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos ")
print_y(f" O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos ")

 ==>  O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4446922 óbitos 
 ==>  O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 1909034 óbitos 
 ==>  O número de óbitos por CID EVITÁVEIS é de 2537888 óbitos 


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [88]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_pop_mun_4merge_temp = pd.read_parquet('files_clean/censo_pop_mun.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 |
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+


In [89]:
# Criar um campo string que concatene ANO, CODMUN_SIM, Sexo e cor
censo_pop_mun_4merge_temp01 = censo_pop_mun_4merge_temp.copy()

censo_pop_mun_4merge_temp01['POP_MERGE'] = '_' + censo_pop_mun_4merge_temp01['Ano'].astype(str) + censo_pop_mun_4merge_temp01['mun_cod_sim']


# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 | POP_MERGE    |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 | _2010_110001 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 | _2010_110002 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 | _2010_110003 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 | _2010_110004 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 | _2010_110005

In [90]:
# Contar quantos valores são repetidos na coluna 'POP_MERGE'
repeated_values = censo_pop_mun_4merge_temp01['POP_MERGE'].value_counts()

# Filtrar os valores que aparecem mais de uma vez
repeated_more_than_once = repeated_values[repeated_values > 1]

# Exibir os valores repetidos
print(repeated_more_than_once)

Series([], Name: count, dtype: int64)


In [91]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files_clean/dobr_evit_merged_v2.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
|  5 | W019       | _500325     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


In [92]:
# Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp01 = dobr_merged_temp.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# Criar um campo string que concatene ANO, CODMUNRES
dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp01['CODMUNRES']

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+-------------+--------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       | ANO_PROXY   | POP_MERGE    |
|----+------------+-------------+-------------+---------------------+-------------+--------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality | _2022       | _2022_292740 |
|  1 | I10        | _355100     | _2019       | Avoidable mortality | _2022       | _2022_355100 |
|  2 | X700       | _500630     | _2019       | Avoidable mortality | _2022       | _2022_500630 |
|  4 | V220       | _500627     | _2019       | Avoidable mortality | _2022       | _2022_500627 |
|  5 | W019       | _500325     | _2019       | Avoidable mortality | _2022       | _2022_500325 |
+----+------------+-------------+-------------+---------------------+-------------+--------------+


In [93]:
# Verificar operação acima
print(f'Anos de ANO_OBITO {dobr_merged_temp01['ANO_OBITO'].unique()}')
print(f'Anos de ANO_PROXY {dobr_merged_temp01['ANO_PROXY'].unique()}')

Anos de ANO_OBITO ['_2019' '_2020' '_2021' '_2022' '_2010']
Anos de ANO_PROXY ['_2022' '_2010']


########## Este bloco de código a seguir retira o CAUSABAS ##########

In [94]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS

col_group= [
            # 'CAUSABAS', 
            'CODMUNRES', 'ANO_OBITO','POP_MERGE']
dobr_merged_temp02 = dobr_merged_temp01.groupby(col_group)['CAUSABAS'].count().reset_index()

# Renomear o campo CAUSABAS para CONTADOR
dobr_merged_temp02.rename(columns={'CAUSABAS':'CONTADOR'}, inplace=True)


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27923
Total de óbitos : 2537888
+----+-------------+-------------+--------------+------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |
|----+-------------+-------------+--------------+------------|
|  0 | _110000     | _2010       | _2010_110000 |         20 |
|  1 | _110000     | _2019       | _2022_110000 |          9 |
|  2 | _110000     | _2020       | _2022_110000 |          7 |
|  3 | _110000     | _2021       | _2022_110000 |          4 |
|  4 | _110000     | _2022       | _2022_110000 |          6 |
+----+-------------+-------------+--------------+------------+


In [95]:
# Verificação dos nomes das colunas
censo_pop_mun_4merge_temp01.columns

Index(['Ano', 'uf_co', 'UF', 'mun_cod_sim', 'mun_no', 'pop_total',
       'pop_under74', 'POP_MERGE'],
      dtype='object')

In [96]:
# Definir campos de censo_pop para merge
censo_pop_mun_4merge_temp02 = censo_pop_mun_4merge_temp01[['POP_MERGE','pop_under74','UF','mun_no']]

# Realizar o merge dos dataframes
dobr_merged_temp03 = pd.merge(dobr_merged_temp02, censo_pop_mun_4merge_temp02, left_on=['POP_MERGE'], right_on=['POP_MERGE'], how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27923
Total de óbitos : 2537888
+----+-------------+-------------+--------------+------------+---------------+------+----------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 |   UF |   mun_no |
|----+-------------+-------------+--------------+------------+---------------+------+----------|
|  0 | _110000     | _2010       | _2010_110000 |         20 |           nan |  nan |      nan |
|  1 | _110000     | _2019       | _2022_110000 |          9 |           nan |  nan |      nan |
|  2 | _110000     | _2020       | _2022_110000 |          7 |           nan |  nan |      nan |
|  3 | _110000     | _2021       | _2022_110000 |          4 |           nan |  nan |      nan |
|  4 | _110000     | _2022       | _2022_110000 |          6 |           nan |  nan |      nan |
+----+-------------+-------------+--------------+------------+---------------+------+----------+


In [97]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp04 = dobr_merged_temp03.dropna(subset=['pop_under74']).reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  4 | _110001     | _2022       | _2022_110001 |         64 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
+----+-------------+------

##### Arquivo tratado final

In [98]:
dobr_merged_temp04.to_parquet('files_clean/dobr_merged_v2.parquet')

In [100]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp04.CONTADOR.sum()
dif = obito_origin - obito_final


print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")
print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")
print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 2537888 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, 4360 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 2533528 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 2 - BR/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* setting: Brasil
* dimension: UF
* subgroup: município

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [3]:
# Define the URL and the destination path
url = "https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm"
destination_path = "downloads_outros/heat_plus_template_validation.xlsm"

# Download the file
response = requests.get(url)
with open(destination_path, 'wb') as file:
    file.write(response.content)

# Note: Enabling macros programmatically is not recommended due to security risks.
# Macros should be enabled manually in Excel settings.

In [101]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_clean/dobr_merged_v2.parquet')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  4 | _110001     | _2022       | _2022_110001 |         64 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
+----+-------------+------

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()

In [102]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_mun'] = (heatds_temp01['CONTADOR']/heatds_temp01['pop_under74'])*100000

# mostrar dados gerais da tabela após transformações
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |   mort_ind_mun |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |        242.191 |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        323.016 |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        327.837 |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        216.951 |
|  4 | _110001     | _

In [105]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun']).sum()
nulos_mort_ind_mun = heatds_temp01['mort_ind_mun'].isnull().sum()


# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
print(f"Valores nulos em 'mort_ind_mun': {nulos_mort_ind_mun}")



# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 0
Valores nulos em 'mort_ind_mun': 0


In [ ]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
# heat_ds_treat2 = heat_ds_treat.copy()

# # Substituir valores infinitos por NaN
# heat_ds_treat2['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# # Rodar se for necessário excluir valores nulos
# heatds_temp02 = heatds_temp01.copy()

# # Excluir registros com NaN na coluna mort_ind_mun
# heatds_temp02.dropna(subset=['mort_ind_mun'], inplace=True)

# # Mudar o nome da coluna mort_ind para mort_ind_mun
# # heat_ds_treat2.rename(columns={'mort_ind':'mort_ind_mun'}, inplace=True)


# # Verificar as primeiras linhas do DataFrame atualizado
# heat_ds_treat1.head()

In [106]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

,CONTADOR,pop_under74,mort_ind_mun
count,27817.000000,2.781700e+04,27817.000000
mean,91.078405,3.464735e+04,268.207663
std,510.470539,1.967627e+05,87.673481
min,1.000000,7.820000e+02,12.640627
25%,13.000000,4.978000e+03,211.782434
50%,27.000000,1.053200e+04,263.208494
75%,62.000000,2.335900e+04,317.028986
max,28299.000000,1.089676e+07,940.733772


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [ ]:
# heat_treat00.columns

In [ ]:
# heat_treat00.pop_under74_uf.sum()

In [ ]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

In [115]:
# Na média ponderada de mortalidade evitável, pegamos os valores de mortalidade para cada município e dividimos pela população, para conseguir o valor para o Brasil e por ano

# Função para calcular a média ponderada
def weighted_avg(group):
    return np.average(group['mort_ind_mun'], weights=group['pop_under74'])

# Agrupar os dados pelo ano e aplicar a função de média ponderada
weighted_avg_per_year = heatds_temp01.groupby('ANO_OBITO').apply(weighted_avg).reset_index()

# Renomear as colunas para melhor entendimento
weighted_avg_per_year.columns = ['ANO_OBITO', 'set_avg_BR']

# Exibir o resultado
print(weighted_avg_per_year)

  ANO_OBITO  set_avg_BR
0     _2010  257.615194
1     _2019  258.919766
2     _2020  256.542023
3     _2021  266.310807
4     _2022  274.720874


/tmp/ipykernel_4975/4059653304.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_per_year = heatds_temp01.groupby('ANO_OBITO').apply(weighted_avg).reset_index()


In [116]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heatds_temp02 = pd.merge(heatds_temp01, weighted_avg_per_year[['ANO_OBITO', 'set_avg_BR']], on='ANO_OBITO', how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+--------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |   mort_ind_mun |   set_avg_BR |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+--------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |        242.191 |      257.615 |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        323.016 |      258.92  |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        327.837 |      256.542 |
|  3 | _110001     | _2021       | _2022_110001 |         45 |  

In [118]:
# Gerar colunas sem informação ou com informação padronizada
# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp03 = heatds_temp02.copy() # caso seja agrupado por UF anteriormente

heatds_temp03['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

heatds_temp03['indicator_abbr'] = 'AVM'
heatds_temp03['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp03['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp03['favourable_indicator'] = 0
heatds_temp03['indicator_scale'] = 100000
heatds_temp03['ordered_dimension'] = 0
heatds_temp03['subgroup_order'] = 0
heatds_temp03['reference_subgroup'] = 0
heatds_temp03['note'] = ''
heatds_temp03['se'] = ''
heatds_temp03['ci_lb'] = ''
heatds_temp03['ci_ub'] = ''
heatds_temp03['iso3'] = 'BRA'

# Renomear colunas de heat_treat01
heatds_temp03 = heatds_temp03.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'UF':'dimension',
    'mun_no':'subgroup',
    'mort_ind_mun':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

# Ajustar coluna de ano
heatds_temp03['date'] = heatds_temp03['date'].str.replace('_','').astype(int)


# mostrar dados gerais da tabela após transformações
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))


Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+--------+--------------+------------+--------------+-------------+----------------------------+------------+-------------------+-----------+------------------+-----------------------------------------------------+----------------------+------------------------+-------------------+---------------------+------------------+----------------------+--------+------+---------+---------+--------+
|    | CODMUNRES   |   date | POP_MERGE    |   CONTADOR |   population | dimension   | subgroup                   |   estimate |   setting_average | setting   | indicator_abbr   | indicator_name                                      | source               |   favourable_indicator |   indicator_scale |   ordered_dimension |   subgroup_order |   reference_subgroup | note   | se   | ci_lb   | ci_ub   | iso3   |
|----+-------------+--------+--------------+------------+--------------+-------------+----------------------------+---------

In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [ ]:
# heat_treat02['subgroup'].unique()

In [ ]:
# heat_treat02.columns

In [120]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  


# Reorder the columns in the DataFrame
heatds_temp04 = heatds_temp03[column_order]


In [123]:
# Transformar estimate e setting_average em int
heatds_temp05 = heatds_temp04.copy()

heatds_temp05['estimate'] = heatds_temp05['estimate'].astype(int)
heatds_temp05['setting_average'] = heatds_temp05['setting_average'].astype(int)

# mostrar dados gerais da tabela após transformações
tab = heatds_temp05
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
+----+-----------+--------+----------------------+------------------+-----------------------------------------------------+-------------+----------------------------+------------+------+---------+---------+--------------+--------+-------------------+--------+------------------------+-------------------+---------------------+------------------+----------------------+
|    | setting   |   date | source               | indicator_abbr   | indicator_name                                      | dimension   | subgroup                   |   estimate | se   | ci_lb   | ci_ub   |   population | note   |   setting_average | iso3   |   favourable_indicator |   indicator_scale |   ordered_dimension |   subgroup_order |   reference_subgroup |
|----+-----------+--------+----------------------+------------------+-----------------------------------------------------+-------------+----------------------------+------------+------+---------+---------+--------------+--------+----

In [124]:
### Arquivo pronto para exportação
heatds_temp05.to_excel('exportar/ds_4val_v2.xlsx', index=False)

In [4]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('downloads_outros/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('exportar/ds_4val_v2.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('exportar/heat_plus_template_ds_4val_v2.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados

# EXCLUIR TUDO A PARTIR DAQUI

In [265]:
teste_fail = pd.read_parquet('files_clean/dobr_merged_clean.parquet')
teste_fail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27916 entries, 3 to 31224
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CODMUNRES    27916 non-null  object 
 1   SEXO         27916 non-null  object 
 2   RACACOR      27916 non-null  object 
 3   ANO          27916 non-null  int64  
 4   POP_MERGE    27916 non-null  object 
 5   obito_count  27916 non-null  int64  
 6   pop_under74  27916 non-null  float64
 7   UF           27916 non-null  object 
 8   mun_nome     27916 non-null  object 
 9   mort_ind     27916 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 2.3+ MB


In [291]:
teste_fail = pd.read_excel('exportar/ds_4val.xlsx')
teste_fail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27879 entries, 0 to 27878
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   setting               27879 non-null  object 
 1   date                  27879 non-null  int64  
 2   source                27879 non-null  object 
 3   indicator_abbr        27879 non-null  object 
 4   indicator_name        27879 non-null  object 
 5   dimension             27879 non-null  object 
 6   subgroup              27879 non-null  object 
 7   estimate              27879 non-null  int64  
 8   se                    0 non-null      float64
 9   ci_lb                 0 non-null      float64
 10  ci_ub                 0 non-null      float64
 11  population            27879 non-null  int64  
 12  note                  0 non-null      float64
 13  setting_average       27879 non-null  int64  
 14  iso3                  0 non-null      float64
 15  favourable_indicato

In [161]:
teste_fail = dobr_merged_pop

In [292]:
# teste_fail0 = teste_fail.loc[teste_fail['Ano'] == 2022]
teste_fail2 = teste_fail.groupby('setting')['population'].sum().reset_index()
# teste_fail2['pop_under74'] = teste_fail2['pop_under74'].astype(int)
teste_fail2.head(10)

,setting,population
0,AC,779569
1,AL,2948673
2,AM,3770310
3,AP,702835
4,BA,13253895
5,CE,8277438
6,DF,2724847
7,ES,3649270
8,GO,6709152
9,MA,6400579


In [266]:
# teste_fail0 = teste_fail.loc[teste_fail['Ano'] == 2022]
teste_fail2 = teste_fail.groupby('UF')['pop_under74'].sum().reset_index()
teste_fail2['pop_under74'] = teste_fail2['pop_under74'].astype(int)
teste_fail2.head(10)

,UF,pop_under74
0,AC,779569
1,AL,2948673
2,AM,3770310
3,AP,702835
4,BA,13253895
5,CE,8277438
6,DF,2724847
7,ES,3649270
8,GO,6709152
9,MA,6400579


In [ ]:
# Calcular a taxa de mortalidade
dobr_merged_clean['mort_ind'] = (dobr_merged_clean['obito_count'] / dobr_merged_clean['pop_under74']) * 100000

# Verificar as primeiras linhas do DataFrame atualizado
dobr_merged_clean.head()

In [91]:
teste_uf_ = censo_pop_clean03[censo_pop_clean03['Nível'] == 'MU']
teste_uf_.head()

,Ano,Nível,Cód.,unidade_no,pop_total,pop_under74,UF,uf_co
27,2010,MU,1100015,Alta Floresta D'Oeste,24392,23948,RO,_11
28,2010,MU,1100023,Ariquemes,90353,89037,RO,_11
29,2010,MU,1100031,Cabixi,6313,6180,RO,_11
30,2010,MU,1100049,Cacoal,78574,76947,RO,_11
31,2010,MU,1100056,Cerejeiras,17029,16646,RO,_11


In [22]:
teste_all_idade = dobr_df.copy()

In [24]:
# campo idade_num formado pelos 2 últimos caracteres de IDADE
teste_all_idade['IDADE_DO_NUM'] = teste_all_idade['IDADE'].str[-2:]

# campo idade_ref formado pelo 1º caractere de IDADE
teste_all_idade['IDADE_DO_REF'] = teste_all_idade['IDADE'].str[0]

In [28]:
teste_all_idade['IDADE_DO_NUM'] = teste_all_idade['IDADE_DO_NUM'].astype(int)
teste_all_idade['IDADE_DO_REF'] = teste_all_idade['IDADE_DO_REF'].astype(int)

In [31]:
teste_all_idade['IDADE_DO_REF'].unique()

array([4, 9, 5, 3, 1, 2, 0])

In [ ]:
teste_all_idade['IDADE_DO_NUM'].unique()

In [20]:
# Fazer uma slice de 30 registros da tabela dobr_df de maneira aleatória
teste_idade = dobr_df[['DTNASC', 'DTOBITO','IDADE']].sample(30)

teste_idade.head()

,DTNASC,DTOBITO,IDADE
505467,26071963,03122022,459
533859,07031941,28012022,480
220070,31031954,28052022,468
1137067,22061934,30072022,488
19522,09011974,25062022,448


In [41]:
# 20 registros aleatórios da tabela dobr_df com IDADE_DO_REF == 0
idade_ref0 = teste_all_idade[teste_all_idade['IDADE_DO_REF'] == 9].sample(20)
teste_idade = idade_ref0[['DTNASC', 'DTOBITO','IDADE']]
teste_idade.head(20)

,DTNASC,DTOBITO,IDADE
519,,31052022,999
1285329,,17092022,999
1346726,,06092022,999
643331,,13062022,999
916924,,10012022,999
351921,,14122022,999
1410064,,19102022,999
1093449,,03112022,999
292407,,18102022,999
13167,,26012022,999


In [43]:
idade_ref9 = teste_all_idade[teste_all_idade['IDADE_DO_REF'] == 9].size
idade_ref9

210330

In [35]:
teste_idade = idade_ref0[['DTNASC', 'DTOBITO','IDADE']]

# campo idade_num formado pelos 2 últimos caracteres de IDADE
teste_idade['IDADE_DO_NUM'] = teste_idade['IDADE'].str[-2:]

# campo idade_ref formado pelo 1º caractere de IDADE
teste_idade['IDADE_DO_REF'] = teste_idade['IDADE'].str[0]

# campo ano_nasc formado pelos 4 últimos caracteres de DTNASC, como número
teste_idade['ANO_NASC'] = teste_idade['DTNASC'].str[-4:]

# campo ano_obito formado pelos 4 últimos caracteres de DTOBITO, como número
teste_idade['ANO_OBITO'] = teste_idade['DTOBITO'].str[-4:]

# mudar formato para número os campos ANO_NASC e ANO_OBITO e IDADE_NUM
teste_idade['IDADE_DO_NUM'] = teste_idade['IDADE_DO_NUM'].astype(int)
teste_idade['ANO_NASC'] = teste_idade['ANO_NASC'].astype(int)
teste_idade['ANO_OBITO'] = teste_idade['ANO_OBITO'].astype(int)

# campo idade_obito formado pela diferença entre ANO_OBITO e ANO_NASC
teste_idade['IDADE_OBITO'] = teste_idade['ANO_OBITO'] - teste_idade['ANO_NASC']

# registrar a diferença entre IDADE_DO_NUM e IDADE_OBITO
teste_idade['DIFERENCA_IDADE_DO'] = teste_idade['IDADE_DO_NUM'] - teste_idade['IDADE_OBITO']

teste_idade.head(20)

/tmp/ipykernel_58425/4162421664.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste_idade['IDADE_DO_NUM'] = teste_idade['IDADE'].str[-2:]
/tmp/ipykernel_58425/4162421664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste_idade['IDADE_DO_REF'] = teste_idade['IDADE'].str[0]
/tmp/ipykernel_58425/4162421664.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,DTNASC,DTOBITO,IDADE,IDADE_DO_NUM,IDADE_DO_REF,ANO_NASC,ANO_OBITO,IDADE_OBITO,DIFERENCA_IDADE_DO
667727,03062022,03062022,023,23,0,2022,2022,0,23
191219,08092022,08092022,042,42,0,2022,2022,0,42
319242,09082022,09082022,020,20,0,2022,2022,0,20
352952,03072022,03072022,035,35,0,2022,2022,0,35
469040,16062022,16062022,040,40,0,2022,2022,0,40
1080834,10072022,10072022,022,22,0,2022,2022,0,22
1264696,19112022,19112022,001,1,0,2022,2022,0,1
485259,01122022,01122022,020,20,0,2022,2022,0,20
1281582,28092022,28092022,005,5,0,2022,2022,0,5
42111,20102022,20102022,010,10,0,2022,2022,0,10


In [21]:
# campo idade_num formado pelos 2 últimos caracteres de IDADE
teste_idade['IDADE_DO_NUM'] = teste_idade['IDADE'].str[-2:]

# campo idade_ref formado pelo 1º caractere de IDADE
teste_idade['IDADE_DO_REF'] = teste_idade['IDADE'].str[0]

# campo ano_nasc formado pelos 4 últimos caracteres de DTNASC, como número
teste_idade['ANO_NASC'] = teste_idade['DTNASC'].str[-4:]

# campo ano_obito formado pelos 4 últimos caracteres de DTOBITO, como número
teste_idade['ANO_OBITO'] = teste_idade['DTOBITO'].str[-4:]

# mudar formato para número os campos ANO_NASC e ANO_OBITO e IDADE_NUM
teste_idade['IDADE_DO_NUM'] = teste_idade['IDADE_DO_NUM'].astype(int)
teste_idade['ANO_NASC'] = teste_idade['ANO_NASC'].astype(int)
teste_idade['ANO_OBITO'] = teste_idade['ANO_OBITO'].astype(int)

# campo idade_obito formado pela diferença entre ANO_OBITO e ANO_NASC
teste_idade['IDADE_OBITO'] = teste_idade['ANO_OBITO'] - teste_idade['ANO_NASC']

# registrar a diferença entre IDADE_DO_NUM e IDADE_OBITO
teste_idade['DIFERENCA_IDADE_DO'] = teste_idade['IDADE_DO_NUM'] - teste_idade['IDADE_OBITO']

teste_idade.head(30)

,DTNASC,DTOBITO,IDADE,IDADE_DO_NUM,IDADE_DO_REF,ANO_NASC,ANO_OBITO,IDADE_OBITO,DIFERENCA_IDADE_DO
505467,26071963,03122022,459,59,4,1963,2022,59,0
533859,07031941,28012022,480,80,4,1941,2022,81,-1
220070,31031954,28052022,468,68,4,1954,2022,68,0
1137067,22061934,30072022,488,88,4,1934,2022,88,0
19522,09011974,25062022,448,48,4,1974,2022,48,0
1082153,22031928,19102022,494,94,4,1928,2022,94,0
667317,17051969,10052022,452,52,4,1969,2022,53,-1
985667,07071932,07062022,489,89,4,1932,2022,90,-1
526522,19101960,13052022,461,61,4,1960,2022,62,-1
1201760,13032003,05022022,418,18,4,2003,2022,19,-1


In [138]:
# Identificar municípios com problemas para merge

# registros com mun_no nulos
mun_nulos = dobr_merged_pop[dobr_merged_pop['mun_no'].isnull()]
mun_nulos.head(30)

,CODMUNRES,ANO_OBITO,POP_MERGE,CONTADOR,pop_under74,UF,mun_no
0,_110000,_2022,_2022_110000,6,NaN,NaN,NaN
53,_120000,_2022,_2022_120000,3,NaN,NaN,NaN
76,_130000,_2022,_2022_130000,26,NaN,NaN,NaN
154,_150000,_2022,_2022_150000,15,NaN,NaN,NaN
454,_210000,_2022,_2022_210000,5,NaN,NaN,NaN
672,_220000,_2022,_2022_220000,4,NaN,NaN,NaN
1081,_240000,_2022,_2022_240000,3,NaN,NaN,NaN
1249,_250000,_2022,_2022_250000,16,NaN,NaN,NaN
1473,_260000,_2022,_2022_260000,50,NaN,NaN,NaN
1659,_270000,_2022,_2022_270000,1,NaN,NaN,NaN


In [142]:
mun_nulos.size

154

In [ ]:
# Calcular a taxa de mortalidade
dobr_merged_final['mort_ind'] = (dobr_merged_final['obito_count'] / dobr_merged_final['pop_under74']) * 100000

# Verificar as primeiras linhas do DataFrame atualizado
dobr_merged_final.head()